In [1]:
# !pip install roboflow

In [2]:
import numpy as np
import pandas as pd
import os
import kagglehub
from roboflow import Roboflow
from sklearn.model_selection import train_test_split
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from google.colab import files

# Import Dataset

In [12]:
# kaggle 데이터 불러오기
path = kagglehub.dataset_download("shubhambaid/skin-burn-dataset")
kg_dir = path

In [13]:
image_label_list = []

valid_exts = ['.jpg', '.jpeg']

for fname in os.listdir(kg_dir):
    if fname.endswith('.txt'):
        base_name = fname.replace('.txt', '')

        # txt와 이름이 같은 img 매칭
        matched_img = None
        for ext in valid_exts:
            candidate = os.path.join(kg_dir, base_name + ext)
            if os.path.exists(candidate):
                matched_img = candidate
                break

        if matched_img is None:
            continue

        # 가장 높은 label 선택
        txt_path = os.path.join(kg_dir, fname)
        with open(txt_path, 'r') as f:
            class_ids = [int(line.split()[0]) for line in f.readlines()]

        if class_ids:
            max_class = max(class_ids)
            image_label_list.append((matched_img, max_class))

In [15]:
# 이미지 수
len(image_label_list)

1223

In [16]:
# roboflow 데이터 불러오기
rf = Roboflow(api_key="tVEtcpzTpsURGFTwfLHZ")
project = rf.workspace("aibuildersclub").project("skin-burns-4yoo2")
version = project.version(2)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...


In [17]:
rf_dir = dataset.location
splits = ["train", "valid", "test"]

# roboflow 데이터 통합
for split in splits:
    split_dir = os.path.join(rf_dir, split)
    csv_path = os.path.join(split_dir, "_classes.csv")

    if not os.path.exists(csv_path):
        continue

    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()  # 열 이름 공백 제거

    for _, row in df.iterrows():
        fname = row['filename'].strip()
        img_path = os.path.join(split_dir, fname)

        if not fname.lower().endswith('.jpg'):
            continue

        if not os.path.exists(img_path):
            continue

        # 클래스 label 추출
        label = int(row[['0', '1', '2']].astype(int).idxmax())
        image_label_list.append((img_path, label))

In [18]:
len(image_label_list)

3848

# Split Dataset

In [19]:
def split_dataset(image_label_list, test_size=0.2, seed=42):
    labels = [label for _, label in image_label_list]
    train_list, val_list = train_test_split(
        image_label_list,
        test_size=test_size,
        stratify=labels,
        random_state=seed
    )
    return train_list, val_list

train_list, val_list = split_dataset(image_label_list)

# Define transform

In [20]:
class SkinBurnDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [21]:
def get_transforms(train=True):
    base = [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]

    if train:
        return transforms.Compose([transforms.RandomHorizontalFlip()] + base)
    else:
        return transforms.Compose(base)

In [22]:
transform_train = get_transforms(train=True)
transform_val = get_transforms(train=False)

# Define dataset

In [23]:
from torch.utils.data import DataLoader

# Dataset 인스턴스 생성
train_dataset = SkinBurnDataset(train_list, transform=transform_train)
val_dataset = SkinBurnDataset(val_list, transform=transform_val)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [24]:
print(f"Train loader: {len(train_loader)} batches")
print(f"Val loader: {len(val_loader)} batches")

Train loader: 97 batches
Val loader: 25 batches


# Import model

In [27]:
def get_model(num_classes=3):
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights)
    in_features = model.classifier[1].in_features # 마지막 classifier만 바꾸기 (3-class 분류용)
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model(num_classes=3).to(device)

# Train model

#### best_model.pt을 현재 디렉토리에 넣고 돌리시면 시간이 단축됩니다!

In [29]:
def train_model(model, train_loader, val_loader, device, epochs=10, lr=1e-4, save_path="best_model.pt"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        avg_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        print(f"Epoch [{epoch+1}/{epochs}]  Loss: {avg_loss:.4f}  Train Acc: {train_acc:.2f}%  Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("saved -", save_path)

    return model

In [30]:
if os.path.exists("best_model.pt"):
    model.load_state_dict(torch.load("best_model.pt", map_location=device))
    model.eval()
    print("weight 로드 완료")
else:
    print("새로 학습 시작")
    model = train_model(model, train_loader, val_loader, device)

weight 로드 완료


# Predict

In [31]:
def predict_image(image_path, model, device):
    image = Image.open(image_path).convert("RGB")
    transform = get_transforms(train=False)
    image_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()


In [32]:
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
pred = predict_image(img_path, model, device)
print(f"예측 클래스: {pred}")

Saving 3도_화상.jpg to 3도_화상.jpg
예측 클래스: 2
